In [43]:
import sh
import os
import datetime
import numpy
import time

#### create the folders that will contain the data
droot = '/home/brandon/repos/bcduino/' # absolute paths
dbash = droot+'bash/'
ddata = droot+'data/'
dpython = droot+'python/'
started = datetime.datetime.now()
tstart = started.strftime("%Y-%m-%d_%H:%M:%S")
dexp = ddata+tstart
dspectra = dexp+'/spectra/'
dtemp = dexp+'/temperature/'
dthermo = dexp+'/thermograph/'
dosc = dexp+'/oscilloscope/'
sh.mkdir(dexp)
sh.mkdir(dspectra)
sh.mkdir(dtemp)
sh.mkdir(dthermo)
sh.mkdir(dosc)
print(tstart)
print(os.listdir(dexp))

### run thermometry - WORKS
stty = sh.Command('/bin/stty')
cat = sh.Command('/bin/cat')
stty('-F','/dev/ttyACM0','57600','raw')
ftemp = dtemp+'temperature_'+tstart
p = cat('/dev/ttyACM0',_bg=True,_out=ftemp)

#### run thermography - WORKS
cp = sh.Command("/bin/cp")
cp(dbash+'thermograph', dthermo)
capturethermograph = sh.Command(dthermo + 'thermograph')
p = capturethermograph(_bg=True,_cwd=dthermo)

#### run spectroscopy - WORKS
# works great, but I don't know how to handle looping at the moment!
#doceanoptics = droot+"c/oceanoptics/seabreeze-3.0.11/SeaBreeze/"
#capturespectrum = sh.Command(doceanoptics+'sample-code/c/seabreeze-util')
#p = capturespectrum("--get-spectrum","--integration-time-us","1000000"
#                    ,_env={"LD_LIBRARY_PATH": doceanoptics+"lib"}
#                    ,_bg=True
#                    ,_out=dspectra+"spectrum_"+datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))
cp(dbash+'spectra.sh', dspectra)
capturespectra = sh.Command(dspectra + 'spectra.sh')
p = capturespectra(dspectra,_bg=True)

#### run oscilloscope
# this is not happening on the Pi3 yet, so don't worry about it
# need to compile another kernel module to make this happen

2016-05-03_03:19:09
['thermograph', 'oscilloscope', 'temperature', 'spectra']


In [48]:
## keep track of how things are going with the logging
loops = 0
maxloops = 2  # total time to loop the status checker
delay = 5     # in seconds
while (loops < maxloops):
    
    runtime = (datetime.datetime.now()-started) #.strftime("%Y-%m-%d_%H:%M:%S")
    print("after running for " + str(runtime).split('.')[0] + ":")
    
    ### check to make sure logging is occurring
    if (len(os.listdir(dspectra)) < 3):
        print("WARNING: spectroscopy down")
    else:
        print(str(len(os.listdir(dspectra))) + " spectra")

    if (len(os.listdir(dthermo)) < 3):
        print("WARNING: thermography down")
    else:
        print(str(len(os.listdir(dthermo))) + " thermographs")

    # not happening on the Pi3 yet
    #if (len(os.listdir(dosc)) < 3):             
    #    print("WARNING: oscilloscope down") 
    #else:
    #    print(len(os.listdir(dosc)) + " oscilloscope traces so far!")

    if (len(numpy.genfromtxt(dtemp+os.listdir(dtemp)[0])) < 10):
        print("WARNING: thermometry down")
    else:
        print(str(len(numpy.genfromtxt(dtemp+os.listdir(dtemp)[0]))) + " temperature datapoints")
        
    print('')
    time.sleep(delay) # give it a few seconds to log some more stuff
    loops = loops + 1


after running for 0:02:23:
40 spectra
138 thermographs
21885 temperature datapoints

after running for 0:02:30:
40 spectra
138 thermographs
21885 temperature datapoints



In [47]:
### stop everything
stop = sh.Command(dbash+'stoprun.sh')
stop()